In [68]:
import torch_geometric
from torch_geometric.data import Data

In [69]:
dir(torch_geometric)

['EdgeIndex',
 'Index',
 'LazyLoader',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_compile',
 '_onnx',
 'backend',
 'compile',
 'contrib',
 'data',
 'datasets',
 'debug',
 'defaultdict',
 'deprecation',
 'device',
 'edge_index',
 'experimental',
 'experimental_mode',
 'explain',
 'get_home_dir',
 'graphgym',
 'home',
 'index',
 'inspector',
 'io',
 'is_compiling',
 'is_debug_enabled',
 'is_experimental_mode_enabled',
 'is_in_onnx_export',
 'is_mps_available',
 'is_torch_instance',
 'is_xpu_available',
 'isinstance',
 'lazy_loader',
 'loader',
 'metrics',
 'nn',
 'profile',
 'resolver',
 'sampler',
 'seed',
 'seed_everything',
 'set_debug',
 'set_experimental_mode',
 'set_home_dir',
 'template',
 'testing',
 'torch',
 'torch_geometric',
 'transforms',
 'typing',
 'utils',
 'visualization',
 'warnings']

In [70]:
import torch

In [71]:
edge_index = torch.tensor([[0, 1, 1, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [72]:
data

Data(x=[3, 1], edge_index=[2, 4])

In [73]:
import torch_geometric
from torch_geometric.datasets import (
    Planetoid,
    CoraFull,
    Amazon,
    Coauthor,
    Reddit,
    WebKB,
    Actor,
    TUDataset,
    QM9,
    MoleculeNet,
)
from torch_geometric.transforms import (
    NormalizeFeatures,
    ToSparseTensor,
    ToUndirected,
    ToUndirected,
    RandomNodeSplit,
    RandomLinkSplit,
    ToSparseTensor,
)
from torch_geometric.nn import (
    GCNConv,
    GATConv,
    SAGEConv,
    ChebConv,
    GINEConv,
    GraphConv,
    TransformerConv,
    TopKPooling,
    global_mean_pool,
    global_max_pool,
    global_add_pool,
)
from torch_geometric.utils import (
    to_undirected,
    from_networkx,
    to_networkx,
    to_dense_batch,
    to_dense_adj,
    to_scipy_sparse_matrix,
    from_scipy_sparse_matrix,
)
from torch_geometric.testing import (
    withPackage,
)
from torch_geometric.home import get_home_dir, set_home_dir

In [74]:
print(data)

Data(x=[3, 1], edge_index=[2, 4])


In [75]:
print(data.keys)

<bound method BaseData.keys of Data(x=[3, 1], edge_index=[2, 4])>


In [76]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 3, 2], [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1], [2]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[4, 1], edge_index=[2, 4])


In [77]:
data.validate(raise_on_error=True)

True

In [78]:
print(data.num_nodes)
print(data.num_edges)
print(data.num_features)
print(data.num_node_features)
print(data.is_directed())
print(data.is_undirected())
print(data.has_isolated_nodes())

4
4
1
1
True
False
False


In [79]:
device = torch.device("cuda")
data = data.to(device)

In [80]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root="/tmp/ENZYMES", name="ENZYMES")
len(dataset)
dataset[0]
dataset.num_classes
dataset.num_node_features

3

In [81]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="/tmp/ENZYMES", name="ENZYMES", use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)
for batch in loader:
    print(batch)
    print(batch.x)
    print(batch.edge_index)
    print(batch.batch)
    break

DataBatch(edge_index=[2, 4048], x=[1063, 21], y=[32], batch=[1063], ptr=[33])
tensor([[11.0000, 15.6121,  0.0000,  ...,  1.0000,  0.0000,  0.0000],
        [ 8.0000, 10.8074,  0.0000,  ...,  1.0000,  0.0000,  0.0000],
        [15.0000, 21.3440,  0.0000,  ...,  1.0000,  0.0000,  0.0000],
        ...,
        [ 8.0000, 23.4656, 27.3900,  ...,  0.0000,  1.0000,  0.0000],
        [ 4.0000,  8.3505,  6.6000,  ...,  0.0000,  1.0000,  0.0000],
        [ 3.0000,  6.1519,  6.9500,  ...,  0.0000,  1.0000,  0.0000]])
tensor([[   0,    0,    0,  ..., 1062, 1062, 1062],
        [   1,    2,   12,  ..., 1040, 1041, 1055]])
tensor([ 0,  0,  0,  ..., 31, 31, 31])


In [82]:
from torch_geometric.utils import scatter
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="/tmp/ENZYMES", name="ENZYMES", use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    print(data)
    print(data.num_graphs)
    x = scatter(data.x, data.batch, dim=0, reduce="mean")
    # mean pooling of node features of each graph (32,21)
    print(x.size())  # (num_graphs, num_node_features)
    # edge_index: (2, num_edges)
    # x: (num_nodes, num_node_features)
    # y: (num_graphs, num_classes)
    # batch: (num_nodes)
    # ptr: (num_graphs + 1), starts and ends of each graph

DataBatch(edge_index=[2, 3808], x=[950, 21], y=[32], batch=[950], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 3640], x=[1001, 21], y=[32], batch=[1001], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 3886], x=[1010, 21], y=[32], batch=[1010], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4774], x=[1346, 21], y=[32], batch=[1346], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 3532], x=[880, 21], y=[32], batch=[880], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4128], x=[1091, 21], y=[32], batch=[1091], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 3790], x=[1014, 21], y=[32], batch=[1014], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4140], x=[1028, 21], y=[32], batch=[1028], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4148], x=[1058, 21], y=[32], batch=[1058], ptr=[33])
32
torch.Size([32, 21])
DataBatch(edge_index=[2, 4172], x=[1102, 21], y=[32], batch=[1102], ptr=[33])
32
torch

In [83]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root="/tmp/ShapeNet", categories=["Airplane"])
dataset[0]

URLError: <urlopen error [SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1006)>

In [ ]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(
    root="/tmp/ShapeNet", categories=["Airplane"], pre_transform=T.KNNGraph(k=6)
)

dataset[0]

In [61]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="/tmp/Cora", name="Cora")
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam, SGD, AdamW, RMSprop, Adagrad, Adadelta
from torch.utils.data import DataLoader, random_split
from torch.nn.modules import Linear, ReLU, Dropout, Sequential, ModuleList
from torch.nn.modules.activation import Sigmoid, Softmax, Tanh

from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss((out[data.train_mask], data.y[data.train_mask]))
    loss.backward()
    optimizer.step()

model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f"Accuracy: {acc:.4f}")

In [ ]:
print(edge_index)
print(edge_index.t())
print(edge_index.t().contiguous())

tensor([[0, 1, 3, 2],
        [1, 0, 2, 1]])
tensor([[0, 1],
        [1, 0],
        [3, 2],
        [2, 1]])
tensor([[0, 1],
        [1, 0],
        [3, 2],
        [2, 1]])


In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="/tmp/IMDB-BINARY", name="IMDB-BINARY", use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)